In [ ]:
!pip install datasets
!pip install evaluate
!pip install sacrebleu

In [ ]:
from datasets import DatasetDict,Dataset
import json
import chardet
import pandas as pd


data=pd.read_csv("hindi_english_parallel.csv", encoding="utf-8", on_bad_lines='skip', encoding_errors="ignore").dropna()
print(data.head())
train = Dataset.from_pandas(data)

dataset = DatasetDict({
    'train': train
})


                                               hindi  \
0    अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें   
1                    एक्सेर्साइसर पहुंचनीयता अन्वेषक   
2              निचले पटल के लिए डिफोल्ट प्लग-इन खाका   
3               ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका   
4  उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...   

                                          english  
0  Give your application an accessibility workout  
1               Accerciser Accessibility Explorer  
2  The default plugin layout for the bottom panel  
3     The default plugin layout for the top panel  
4  A list of plugins that are disabled by default  


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['hindi', 'english', '__index_level_0__'],
        num_rows: 48898
    })
})

In [ ]:
dataset=dataset['train'].train_test_split(0.2)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['hindi', 'english', '__index_level_0__'],
        num_rows: 39118
    })
    test: Dataset({
        features: ['hindi', 'english', '__index_level_0__'],
        num_rows: 9780
    })
})

In [ ]:
dataset['train'][0]

{'hindi': 'प्रोग्राम', 'english': 'Program', '__index_level_0__': 6093}

In [ ]:
from transformers import AutoTokenizer, MarianTokenizer, TFMarianMTModel

checkpoint = 'Helsinki-NLP/opus-mt-en-hi'
tokenizer = MarianTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
# # Check if the words are in the vocabulary

# vocab = tokenizer.get_vocab()

# word1 = ""

# if word1 in vocab:
#     print(f"'{word1}' is in the vocabulary.")
# else:
#     print(f"'{word1}' is NOT in the vocabulary.")

In [ ]:
source_lang='english'
target_lang='hindi'
# prefix='translate English to Hindi:'
def preprocess_text(examples):
  inputs=[example for example in examples[source_lang]]
  targets=[example for example in examples[target_lang]]
  tokenized_inputs=tokenizer(inputs,max_length=128,truncation=True)
  with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)
  tokenized_inputs["labels"] = labels["input_ids"]
  return tokenized_inputs

In [ ]:
preprocessed_dataset=dataset.map(preprocess_text,batched=True)

Map:   0%|          | 0/39118 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/9780 [00:00<?, ? examples/s]

In [ ]:
preprocessed_dataset['train'][1]

# tokenizer.decode(preprocessed_dataset['train'][1]["labels"],skip_special_tokens=True)

{'hindi': 'पंक्ति संख्या प्रदर्शित करें',
 'english': 'Show / Hide marker margin',
 '__index_level_0__': 6459,
 'input_ids': [515, 351, 4481, 20567, 14607, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1],
 'labels': [1637, 877, 2001, 84, 0]}

In [ ]:
# from transformers import DataCollatorForSeq2Seq
# datacollator=DataCollatorForSeq2Seq(tokenizer=tokenizer,model="Helsinki-NLP/opus-mt-en-de",return_tensors='tf')

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model=TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
import tensorflow as tf

input_ids = [example['input_ids'] for example in preprocessed_dataset['train']]
labels = [example['labels'] for example in preprocessed_dataset['train']]

max_length = max(len(seq) for seq in input_ids)
input_ids = tf.keras.preprocessing.sequence.pad_sequences(input_ids, maxlen=max_length, padding='post', truncating='post')
labels = tf.keras.preprocessing.sequence.pad_sequences(labels, maxlen=max_length, padding='post', truncating='post')

input_ids_tensor = tf.constant(input_ids, dtype=tf.int64)
labels_tensor = tf.constant(labels, dtype=tf.int64)
train_set = tf.data.Dataset.from_tensor_slices((input_ids_tensor, labels_tensor))
train_set = train_set.shuffle(buffer_size=len(input_ids)).batch(16)

# input_ids_tensor = tf.constant(input_ids, dtype=tf.int64)
# labels_tensor = tf.constant(labels, dtype=tf.int64)




In [ ]:
import tensorflow as tf

input_ids = [example['input_ids'] for example in preprocessed_dataset['test']]
labels = [example['labels'] for example in preprocessed_dataset['test']]

input_ids = tf.keras.preprocessing.sequence.pad_sequences(input_ids, maxlen=max_length, padding='post', truncating='post')
labels = tf.keras.preprocessing.sequence.pad_sequences(labels, maxlen=max_length, padding='post', truncating='post')
input_ids_tensor = tf.constant(input_ids, dtype=tf.int64)
labels_tensor = tf.constant(labels, dtype=tf.int64)
val_set = tf.data.Dataset.from_tensor_slices((input_ids_tensor, labels_tensor))
val_set = val_set.shuffle(buffer_size=len(input_ids)).batch(16)




In [ ]:
# train_set=model.prepare_tf_dataset(
#     preprocessed_dataset['train'],
#     batch_size=16,
#     shuffle=True,
#     collate_fn=datacollator
# )
# val_set=train_set=model.prepare_tf_dataset(
#     preprocessed_dataset['test'],
#     batch_size=16,
#     shuffle=False,
#     collate_fn=datacollator
# )

In [ ]:
from tensorflow import keras
model.compile(keras.optimizers.Adam(2e-5))

In [ ]:
import evaluate
metric=evaluate.load('sacrebleu')

In [ ]:
import numpy as np
def compute_metrics(p):
  preds,labels=p
  if isinstance(preds,tuple):
    preds=preds[0]
  decoded_preds=tokenizer.decode_batch(preds,skip_special_tokens=True)
  decoded_preds=[preds.strip() for preds in decoded_preds]
  labels=np.where(labels!=-100,labels,tokenizer.pad_token_id)
  decoded_labels=tokenizer.decode_batch(labels,skip_special_tokens=True)
  decoded_labels=[[label.strip()] for label in decoded_labels]
  result=metric.compute(predictions=decoded_preds,references=decoded_labels)
  result={'bleu':result['score']}
  predicted_lens=[np.count_nonzero(pred!=tokenizer.pad_token_id) for pred in preds]
  result['gen_len']=np.mean(predicted_lens)
  return result


In [ ]:
from transformers.keras_callbacks import KerasMetricCallback
callback=KerasMetricCallback(metric_fn=compute_metrics,eval_dataset=val_set,predict_with_generate=True,use_xla_generation=True,
    generate_kwargs={"max_length": 128})

In [ ]:
model.fit(train_set,epochs=3)

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2445/2445 [==============================] - 1174s 438ms/step - loss: 0.0449
Epoch 2/3
2445/2445 [==============================] - 1076s 440ms/step - loss: 0.0155
Epoch 3/3
2445/2445 [==============================] - 1078s 441ms/step - loss: 0.0096


In [ ]:
input_text='sir give full marks, please (:'
input_ids=tokenizer(input_text,return_tensors='tf').input_ids
output=model.generate(input_ids)
tokenizer.decode(output[0],skip_special_tokens=True)

'सर सर पूरा निशान दे, कृपया भरें निशान दे दो, कृपया (G: 09:'

In [ ]:
evaluation_result = model.evaluate(val_set)

print("Evaluation result:", evaluation_result)

612/612 [==============================] - 123s 175ms/step - loss: 0.0086
Evaluation result: 0.0085778022184968


In [ ]:
"DONE"

'DONE'